In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import pickle
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns

In [2]:
data = pd.read_csv("../data/Language Detection.csv")

In [3]:
data.Language.value_counts()

English       1385
French        1014
Spanish        819
Portugeese     739
Italian        698
Russian        692
Sweedish       676
Malayalam      594
Dutch          546
Arabic         536
Turkish        474
German         470
Tamil          469
Danish         428
Kannada        369
Greek          365
Hindi           63
Name: Language, dtype: int64

In [4]:
data.iloc[0].Text

' Nature, in the broadest sense, is the natural, physical, material world or universe.'

In [5]:
# 'English'
data = data[data.Language.isin(['French', 'Russian'])]

In [6]:
data.shape

(1706, 2)

In [7]:
data.Language.value_counts()

French     1014
Russian     692
Name: Language, dtype: int64

In [17]:
def length(text):
    words = text.split()
    return len(words)

In [9]:
def is_ru(col):
    return 1 if col == "Russian" else 0

In [10]:
def clean(text):
    # remove numbers in text
    text = re.sub(r'[!@#$(),\n"%^*?\:;~`0-9]', ' ', text)
    # remove other symbols in text
    text = re.sub(r'[[]]', ' ', text)
    text = text.lower()
    return text

In [11]:
data['clean'] = data['Text'].apply(clean)

/Users/akerke/Desktop/Masterstudium/4.Semester/MasterThesis/war_and_peace/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Possible nested set at position 1
  """


In [12]:
data['lang_code'] = data['Language'].apply(is_ru)

In [18]:
data['len'] = data['Text'].apply(length)

In [19]:
data.head()

,Text,Language,clean,lang_code,len
3250,Si vous disposez d'ouvrages ou d'articles de r...,French,si vous disposez d'ouvrages ou d'articles de r...,0,54
3251,Comment ajouter mes sources ?,French,comment ajouter mes sources,0,5
3252,Cette page ou section est en train d'être trad...,French,cette page ou section est en train d'être trad...,0,40
3253,Vous pouvez aider au développement de Wikipédi...,French,vous pouvez aider au développement de wikipédi...,0,16
3254,Le mot nature est un terme polysémique (c’est-...,French,le mot nature est un terme polysémique c’est-...,0,50


In [20]:
data.tail()

,Text,Language,clean,lang_code,len
6681,Однажды Мелли и Терри снова пришли встретиться...,Russian,однажды мелли и терри снова пришли встретиться...,1,9
6682,"О, привет, вы двое, так скажите нам Мэриан.",Russian,о привет вы двое так скажите нам мэриан.,1,8
6683,Как' теперь нарциссизм Мэриан рассказал им обо...,Russian,как' теперь нарциссизм мэриан рассказал им обо...,1,18
6684,"Думаю, она не хотела бы больше золотого хлеба,...",Russian,думаю она не хотела бы больше золотого хлеба ...,1,11
6685,Терри вы на самом деле немного похожи на этого...,Russian,терри вы на самом деле немного похожи на этого...,1,23


In [21]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [22]:
X = cv.fit_transform(data["clean"])

In [28]:
y = data["lang_code"]

In [29]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, y, test_size = 0.20)

In [30]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()

In [31]:
clf.fit(Xtrain, Ytrain)

MultinomialNB()

In [32]:
Ypred = clf.predict(Xtest)

In [33]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [34]:
accr = accuracy_score(Ytest, Ypred)
print(accr)

0.9941520467836257


In [35]:
print(classification_report(Ytest, Ypred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       213
           1       0.99      0.99      0.99       129

    accuracy                           0.99       342
   macro avg       0.99      0.99      0.99       342
weighted avg       0.99      0.99      0.99       342



In [36]:
cm = confusion_matrix(Ytest, Ypred)

In [37]:
languages = {0:"french", 1:"russian"}

In [38]:
x = cv.transform(["Eh bien, mon prince"])
print(clf.predict_proba(x))
print(clf.predict(x))

[[0.99793342 0.00206658]]
[0]


In [45]:
def predict(text):
    x = cv.transform([text])
    lang = clf.predict(x)
    probs = clf.predict_proba(x)
    print("Probability is: ", probs[0][0])
    print("The language is", languages[lang[0]])

In [46]:
predict("Eh bien, mon prince")

Probability is:  0.9979334150015573
The language is french


In [47]:
predict("Анна Павловна кашляла несколько дней, у нее был  грипп , как она говорила  (грипп  был тогда новое слово, употреблявшееся только редкими)")

Probability is:  1.8257416471005837e-12
The language is russian


In [48]:
predict("В записочках, разосланных утром с красным лакеем, было написано без различия во всех:   «Si vous n’avez rien de mieux à faire, M. le comte (или mon prince), et si la perspective de passer la soirée chez une pauvre malade ne vous effraye pas trop, je serai charmée de vous voir chez moi entre 7 et 10 heures.")

Probability is:  1.0
The language is french


In [49]:
predict("Je vois que je vous fais peur,  садитесь и рассказывайте.")

Probability is:  0.999999999579245
The language is french


In [50]:
predict("— Вы весь вечер у меня, надеюсь?")

Probability is:  7.408074274483916e-06
The language is russian


In [51]:
predict("— Le général Koutouzoff, — сказал Болконский, ударяя на последнем слоге  zoff , как француз, — a bien voulu de moi pour aide-de-camp...   — Et Lise, votre femme?")

Probability is:  0.9999999999953957
The language is french


In [60]:
import spacy
from spacy_langdetect import LanguageDetector
from spacy.language import Language

def create_lang_detector(nlp, name):
    return LanguageDetector()

Language.factory("language_detector", func=create_lang_detector)

# nlp = spacy.load("en_core_web_sm")
nlp = spacy.load("fr_core_news_sm")
nlp.add_pipe('language_detector')

ValueError: [E004] Can't set up pipeline component: a factory for 'language_detector' already exists. Existing factory: <function create_lang_detector at 0x11b953170>. New factory: <function create_lang_detector at 0x1399becb0>

In [80]:
text = 'This is an english text.'
doc = nlp(text)
print(doc._.language)
for sent in doc.sents:
    print(sent, sent._.language)

NameError: name 'nlp' is not defined

In [54]:
text2 = "Eh bien, mon prince"
doc = nlp(text2)
print(doc._.language)

{'language': 'es', 'score': 0.47201035653537604}


In [15]:
text3 = "Анна Павловна кашляла несколько дней, у нее был  грипп , как она говорила  (грипп  был тогда новое слово, употреблявшееся только редкими)"

In [16]:
doc = nlp(text3)
print(doc._.language)

{'language': 'ru', 'score': 0.9999987876644919}


In [17]:
text4 = "Je vois que je vous fais peur,  садитесь и рассказывайте."
doc = nlp(text4)
print(doc._.language)

{'language': 'ru', 'score': 0.42856904927809947}


#### Create own CountVectorizor

In [65]:
data = [
    "doubt thou the stars are fire",
    "doubt that the sun doth move;",
    "doubt truth to be a liar;",
    "but never doubt i love."
]

In [81]:
def cv_fit(data) -> int:
    """return index for given text"""
    unique_words = set()
    for sentence in data:
        for word in sentence.split(" "):
            word = re.sub(r'[!@#$(),\n"%^*?\:.;~`0-9]', '', word)
            if len(word) >= 2:
                unique_words.add(word)
    vocab = {}
    for index, word in enumerate(sorted(list(unique_words))):
        vocab[word] = index
    return vocab

In [85]:
vocabulary = cv_fit(data)

In [86]:
vocabulary

{'are': 0,
 'be': 1,
 'but': 2,
 'doth': 3,
 'doubt': 4,
 'fire': 5,
 'liar': 6,
 'love': 7,
 'move': 8,
 'never': 9,
 'stars': 10,
 'sun': 11,
 'that': 12,
 'the': 13,
 'thou': 14,
 'to': 15,
 'truth': 16}

In [42]:
from collections import Counter
from scipy.sparse import csr_matrix

In [94]:
def cv_fit_transform(data):
    vocabulary = cv_fit(data)
    row, col, val = [],[],[]
    for sentence_id, sentence in enumerate(data):
        count_word = dict(Counter(sentence.split(" ")))
        for word, count in count_word.items():
            word = re.sub(r'[!@#$(),\n"%^*?\:.;~`0-9]', '', word)
            if len(word) >= 2:
                col_index = vocabulary[word]
                row.append(sentence_id)
                col.append(col_index)
                val.append(count)
    return csr_matrix( (val, (row, col)) , shape=(len(data), len(vocabulary)))

In [95]:
cv_fit_transform(data).toarray()

array([[1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0],
       [0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]])

In [96]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [97]:
X = cv.fit_transform(data)
X.todense()

matrix([[1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0],
        [0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0],
        [0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
        [0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]])

In [98]:
cv_fit_transform(data).toarray() == X.todense()

matrix([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True]])